<center><h1> FULL CODE BOARD MONITOR TOOL </center></h1> 

In [7]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as wait
import pandas as pd
import csv
from datetime import datetime, timedelta

## I- Getting the initial database

### A- Getting all links iterating through pages

In [ ]:
last_page_number = 514

def get_links():
    url = 'https://www.ge.ch/recherche-entreprises-dans-registre-du-commerce-geneve'
    ## GETTING THE FIRST PAGE
    driver = webdriver.Chrome(ChromeDriverManager().install())    
    driver.get(url)
    driver.switch_to.frame(driver.find_element_by_name('mainFrame'));


    ## MOVING TO THE SECOND PAGE
    button = driver.find_element_by_id("companySearch_display")
    if button:
            button.click()
            WebDriverWait(driver, 3)



    ## ITERATING THROUGH ALL RESPONSE PAGES TO GET HREFs       
    more_pages = True
    a = []
    p = 0

    while more_pages == True:
        table = driver.find_element_by_class_name("result")

        i = 0
        for row in table.find_elements_by_css_selector('tr'):
            for cell in row.find_elements_by_css_selector('td'):
                try:
                    content = cell.find_element_by_css_selector("a[href*='companyReport']")
                    link = content.get_attribute('href')
                    a.append(link)
                    print(link)
                    print('all good at row ', i, ' page ', p )
                    i+=1

                except Exception as e:
                    i+=1
                    continue
        p += 1

        suivant = driver.find_elements_by_xpath("//*[contains(text(), 'Suivant')]")
        if p < last_page_number:
                more_pages = True
                suivant[0].click()
        else:
            more_pages = False

    return a

links = a[:51424]
        

### B- Getting all board members names

In [ ]:
def companies_and_boards(links):
    driver = webdriver.Chrome(ChromeDriverManager().install())     
    n = 0
    dictionnary = {}
    name = 0

    for link in links:
        driver.get(link)

        values = []
        keys = []
        try:
            table_of_interest = driver.find_element_by_xpath("//*[contains(text(), 'Raison')]")
            parent_table = table_of_interest.find_element_by_xpath('..')
            parent_table2 = parent_table.find_element_by_xpath('..')
            check = parent_table2.find_elements_by_css_selector("td[style='text-decoration: none;']");
            name = check[0].text

            print('got name', name)

        except Exception as e:
            pass


        try:
            table_of_interest = driver.find_element_by_xpath("//*[contains(text(), 'Nom')]")
            parent_table = table_of_interest.find_element_by_xpath('..')
            parent_table2 = parent_table.find_element_by_xpath('..')
            check = parent_table2.find_elements_by_css_selector("td[style='text-decoration: none;']");
            name_company = check[0].text


        except Exception as e:
            name_company = 'unavailable'
            pass



        try:
            table_of_interest = driver.find_element_by_xpath("//*[contains(text(), 'qualité pour signer')]")
            parent_table = table_of_interest.find_element_by_xpath('..')
            parent_table2 = parent_table.find_element_by_xpath('..')

            for colname in parent_table2.find_elements_by_css_selector('th'):
                keys.append(colname.text)

            for row in parent_table2.find_elements_by_css_selector('tr'):
                for d in row.find_elements_by_css_selector('td'):
                    values.append(d.text)

            keys= keys[1:]
            values.append(name_company)
            print('got board members for company ', name)


            i = 0
            m = 0

            dictionnary['header'] = keys
            while i < len(keys):
                key = keys[i]
                up_to = m+len(keys)
                data = values[m:int(up_to)]
                if data :
                    dictionnary[str(name)] = data
                m += 3
                i += 1
                name +=1

            n +=1


        except Exception as e:
            continue
            n+=1
            name +=1
            
    return dictionnary

In [ ]:
results = companies_and_boards(links)
df = pd.DataFrame.from_records(result)
df

## II- Getting updates each week

In [1]:
def get_update():
    url = 'https://www.ge.ch/recherche-entreprises-dans-registre-du-commerce-geneve'
    driver = webdriver.Chrome(ChromeDriverManager().install())    
    driver.get(url)
    driver.switch_to.frame(driver.find_element_by_name('mainFrame'));
    advanced_search = driver.find_element_by_xpath("//*[contains(text(),'+ Critères avancés de recherche')]")
    advanced_search.click()

    from selenium.webdriver.support.ui import WebDriverWait

    last_week = datetime.now() - timedelta(7)
    last_week = datetime.strftime(last_week, '%d.%m.%Y')
    last_week
    driver.find_element_by_name("originDateBegin").send_keys(str(last_week))
    WebDriverWait(driver, 3)
    search = driver.find_element_by_id("companySearch_display")
    driver.execute_script("arguments[0].click();", search)
    
    ## ITERATING THROUGH ALL RESPONSE PAGES TO GET HREFs       
    more_pages = True
    a = []
    p = 0

    while more_pages == True:
        table = driver.find_element_by_class_name("result")

        i = 0
        for row in table.find_elements_by_css_selector('tr'):
            for cell in row.find_elements_by_css_selector('td'):
                try:
                    content = cell.find_element_by_css_selector("a[href*='companyReport']")
                    link = content.get_attribute('href')
                    a.append(link)
                    print(link)
                    print('all good at row ', i, ' page ', p )
                    i+=1

                except Exception as e:
                    i+=1
                    continue
        p += 1

        suivant = driver.find_elements_by_xpath("//*[contains(text(), 'Suivant')]")
        if p < last_page_number:
                more_pages = True
                suivant[0].click()
        else:
            more_pages = False

    links = a
    companies_and_boards(links)
    
    return dictionnary

results = get_update()
df = pd.DataFrame.from_records(result)
df

NameError: name 'datetime' is not defined

## III - Data cleaning and wrangling

In [37]:
df = pd.read_excel('second_person_company_df.xlsx')

In [39]:
df2 = df.set_axis(['Index', 'Name', 'Status', 'Signature mode', 'Company'], axis=1, inplace=False)

try:
    df2 = df.set_axis(['Index','Name', 'Status', 'Signature mode', 'Company'], axis=1, inplace=False)
except:
    df2 = df.set_axis(['Index','Name', 'Status', 'Signature mode', 'Company'], axis=1, inplace=False)

df3 = df2["Name"].str.split(',', expand=True)
try:
    df3 = df3.set_axis(['Name and Surname', 'Origin', 'To', 'For 1', 'For 2', 'For 3' ], axis=1, inplace=False)
except:
    df3 = df3.set_axis(['Name and Surname', 'Origin', 'To', 'For 1', 'For 2' ], axis=1, inplace=False)

result = pd.concat([df3, df2], axis=1)
result_epurated = result[['Name and Surname', 'Origin', 'To','Status', 'Signature mode','Company']]


df = result_epurated.drop_duplicates()

## Remove code
df['Name and Surname'] = df['Name and Surname'].astype(str)
df['Name and Surname'].mask((df['Name and Surname'].str.contains("CHE-")),(df['Name and Surname'].str[:-18]), inplace=True)
df['Name and Surname'].mask((df['Name and Surname'].str.contains("CH-")),(df['Name and Surname'].str[:-19]), inplace=True)

df = df[df['Name and Surname'] != '* du conseil']
df = df[df['Name and Surname'] != '* du conseil de fondation']
df = df[df['Name and Surname'] != '* du comité']
df = df[df['Name and Surname'] != '*du comité']
df = df[df['Name and Surname'] != '* du comité']
df = df[df['Name and Surname'] != '*du conseil de fondation']


/var/folders/vn/d12r03rj2tq6sp8nd5k9209h0000gn/T/ipykernel_3918/717626587.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name and Surname'] = df['Name and Surname'].astype(str)
/var/folders/vn/d12r03rj2tq6sp8nd5k9209h0000gn/T/ipykernel_3918/717626587.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name and Surname'].mask((df['Name and Surname'].str.contains("CHE-")),(df['Name and Surname'].str[:-18]), inplace=True)
/var/folders/vn/d12r03rj2tq6sp8nd5k9209h0000gn/T/ipykernel_3918/717626587.py:21: SettingWithCopyWarning: 
A value is trying to be s

#### Wrangling to allow for network analysis

In [43]:
# Add number of occurence
df['Weight_Person'] = df.groupby('Name and Surname')['Name and Surname'].transform('size')
df['Weight_Company'] = df.groupby('Company')['Company'].transform('size')

## Keep relevant columns & remove duplicates
df = df[['Name and Surname','Company','Weight_Person','Weight_Company']]
df =df.drop_duplicates()

,Name and Surname,Company,Weight_Person,Weight_Company
0,Dudouit Elkaïm Rachel,CMP Cercle Menus Plaisirs Sàrl,1,2.0
1,Herr-Dudouit Sephora,CMP Cercle Menus Plaisirs Sàrl,1,2.0
2,Eretzian Jean-Philippe,CHAUFFAGE CONCEPT Sàrl,1,3.0
3,Büchi Thomas,CHAUFFAGE CONCEPT Sàrl,1,3.0
4,Experaudit SA,CHAUFFAGE CONCEPT Sàrl,10,3.0
...,...,...,...,...
22349,Da Soledade Mendes Irène da Conceiçao,Echaloc SA,1,4.0
22350,Dembinsky Pawel,Association Internationale Pour l'Enseignement...,1,4.0
22351,Buttet Nicolas,Association Internationale Pour l'Enseignement...,1,4.0
22352,Loze Bertrand,Association Internationale Pour l'Enseignement...,1,4.0


In [ ]:
def get_data_js(df_graph):
    df_graph = df_graph[['Name and Surname', 'company_name']]
    df_graph.columns = ['source','target']
    grouped_src_dst = df_graph.groupby(["source","target"]).size().reset_index()
    
    unique_nodes = pd.Index(grouped_src_dst['source']
                      .append(grouped_src_dst['target'])
                      .reset_index(drop=True).unique())

    nodes_list = []
    for ip in unique_nodes:
        nodes_list.append({"name":ip})
    nodes_list

    df1 = pd.DataFrame(nodes_list)


    yala = []
    i = 0
    while i < len(grouped_src_dst):
        yala.append({"group":1, "name":grouped_src_dst.iloc[i]['source']})
        yala.append({"group":2, "name":grouped_src_dst.iloc[i]['target']})
        i +=1

    df2 = pd.DataFrame(yala)
    result = pd.merge(df1, df2, on=["name", "name"])
    check = df1.merge(df2, how='left', on='name').drop_duplicates()
    
    nodes_list = []

    i = 0 
    while i < len(check):
        nodes_list.append({"group":str(check.iloc[i]['group']), "name":str(check.iloc[i]['name'])})
        #nodes_list.append({"name":str(check.iloc[i]['name'])})
        i += 1
        
    grouped_src_dst.rename(columns={0:'count'}, inplace=True)
    temp_links_list = list(grouped_src_dst.apply(lambda row: {"source": row['source'], "target": row['target'], "value": row['count']}, axis=1))

    links_list = []
    for link in temp_links_list:
        record = {"source":unique_nodes.get_loc(link['source']),
         "target": unique_nodes.get_loc(link['target'])}
        links_list.append(record)
        
    return nodes_list, links_list

## IV- Creating information, answering questions

### a. Most direct connections

#### 1. As board members (=number of companies in which appear)

Top ranking

In [87]:
counts = pd.DataFrame(df['Name and Surname'].value_counts())

Who are those connections

In [88]:
y = input()

Herr-Dudouit Sephora


In [89]:
connections = df[df['Name and Surname'] == y]
connections

,Name and Surname,Company,Weight_Person,Weight_Company
1,Herr-Dudouit Sephora,CMP Cercle Menus Plaisirs Sàrl,1,2.0


#### 2. As company (=number of board members)

In [90]:
y = input()

CMP Cercle Menus Plaisirs Sàrl


In [79]:
connections = pd.DataFrame(df['Company'].value_counts())

In [91]:
connections = df[df['Name and Surname'] == y]
connections

,Name and Surname,Company,Weight_Person,Weight_Company


### b. Is a connected to b? If yes by how many degrees?

In [56]:
x = input()

In [57]:
y = input()

In [ ]:
def get_full_connectivity_company(df, y):
    
    ## CASE 2= NOT FOUND ON BOARD BUT IN REGISTER
    ## 1. Find board members
    more_members = (df[df['Company']==str(y)]['Name and Surname']).tolist()
    
    
    board_members = []
    companies = []
    condition = False
    
    while (condition == False):
        
        more_companies = []
        
        ## 2. Search if board members belong to more companies
        for i in more_members:
            find_as_company = (df[df['Name and Surname']==str(i)])['Company'].tolist()
            more_companies = more_companies + find_as_company
            # remove duplicates
            more_companies = list(dict.fromkeys(more_companies))
       
        more_members = []
        
        for i in more_companies:
            find_as_company = (df[df['Company']==str(i)])['Name and Surname'].tolist()
            more_members = more_members + find_as_company
            # remove duplicates
            more_members = list(dict.fromkeys(more_members))
            
        
        condition1 = (more_companies == companies) 
        condition2 = (more_members == board_members)
        condition = condition1 and condition2
        
        companies = more_companies
        board_members = more_members

    board_members = pd.DataFrame(board_members)
    companies = pd.DataFrame(companies)
    board_members.columns = ['member_name']
    companies.columns = ['company_name']

    df_graph = pd.merge(df, companies, how='inner',left_on='Company', right_on='company_name')

    return df_graph, board_members, companies


In [ ]:
def check_if_connected(x,y):
    board_members, companies = get_full_connectivity_company(x)
    board_members = pd.DataFrame(board_members)
    companies = pd.DataFrame(companies)
    board_members.columns = ['member_name']
    companies.columns = ['company_name']

    network_y = pd.merge(df, board_members, how='inner',left_on='Name and Surname', right_on='member_name')

    check = network_y[(network_y['Name and Surname'] == str(y))|(network_y['Company'] == str(y))]

    if len(check) > 0:
        print("They are connected!")
        create_graph(network_y)
        
    else:
        print('They are not connected.')

## IV- Visualising

### A- Option 1 - Pyvis

In [ ]:
from pyvis.network import Network

def create_graph_person(df_graph):
    net = Network()

    i = 0
    while i < len(df_graph):
        net.add_node(str(df_graph.iloc[i]['Name and Surname']), label=str(df_graph.iloc[i]['Name and Surname']))
        net.add_node(str(df_graph.iloc[i]['Company']), label=str(df_graph.iloc[i]['Company']))

        i += 1

    i = 0
    while i < len(df_graph):
        net.add_edge(str(df_graph.iloc[i]['Name and Surname']), str(df_graph.iloc[i]['Company']))
        i +=1

    net.show_buttons(filter_=True)
    net.show('graph_person.html')



### B- Option 2- D3.js

SEE VISUAL STUDIO CODE.